# Sankey Income and Budget Tracker

## Imports

In [1]:
import datetime
import os
from pathlib import Path

import dotenv
import pandas as pd
import plotly.graph_objects as go

dotenv.load_dotenv()

True

## Setup

Sankey diagrams are basically fancy graphs that show the flow of data, money, or
anything into sub-buckets. Generally, the charts flow in only one direction, so
these could be considered directed acyclic graphs (DAG).

See [sankeymatic](https://sankeymatic.com) for an example. They use a DSL
similar to the following.

```text
nodeA -> [label1,value1] -> nodeB
nodeB -> [label2,value2] -> nodeC
...
```

[See this answer on how to format the
DataFrame](https://stackoverflow.com/a/55468928) for the Sankey diagrams.

In [2]:
node_df = pd.DataFrame([], columns=["Label", "Color"])
link_df = pd.DataFrame([], columns=["Source", "Target", "Value", "Link Color"])

In [3]:
def add_node_label(
    df: pd.DataFrame,
    label: str,
    color: str,
) -> None:
    df.loc[len(df)] = {"Label": label, "Color": color}


def add_node_connection(
    node_df: pd.DataFrame,
    link_df: pd.DataFrame,
    source: str,
    target: str,
    value: float,
) -> None:
    # TODO: should we handle this error?
    source_idx = node_df.loc[node_df["Label"] == source].index.item()
    target_idx = node_df.loc[node_df["Label"] == target].index.item()

    link_df.loc[len(link_df)] = {
        "Source": source_idx,
        "Target": target_idx,
        "Value": value,
    }

## Income

Add any other sources of income than primary job

### Salary

In [4]:
weekly_salary = float(os.getenv("WEEKLY_SALARY"))
monthly_salary = weekly_salary * 4

weekly_net_pay = float(os.getenv("WEEKLY_NET_PAY"))
monthly_net_pay = weekly_net_pay * 4  # sadge

In [5]:
salary_color = "blue"

# Add the node labels
add_node_label(node_df, label="Salary", color=salary_color)
add_node_label(node_df, label="Income", color=salary_color)

add_node_connection(
    node_df,
    link_df,
    source="Salary",
    target="Income",
    value=monthly_salary,
)

add_node_label(node_df, label="Net Income", color=salary_color)
add_node_connection(
    node_df,
    link_df,
    source="Income",
    target="Net Income",
    value=monthly_net_pay,
)

## Taxes

In [6]:
weekly_taxes = float(os.getenv("WEEKLY_TAXES"))
monthly_taxes = weekly_taxes * 4

weekly_taxes_fed_witholding = float(os.getenv("WEEKLY_TAXES_FED_WITHOLDING"))
monthly_taxes_fed_witholding = weekly_taxes_fed_witholding * 4

weekly_taxes_fed_fica = float(os.getenv("WEEKLY_TAXES_FED_FICA"))
monthly_taxes_fed_fica = weekly_taxes_fed_fica * 4

weekly_taxes_fed_oasdi = float(os.getenv("WEEKLY_TAXES_FED_OASDI"))
monthly_taxes_fed_oasdi = weekly_taxes_fed_oasdi * 4

weekly_taxes_co_leave = float(os.getenv("WEEKLY_TAXES_CO_LEAVE"))
monthly_taxes_co_leave = weekly_taxes_co_leave * 4

weekly_taxes_co_witholding = float(os.getenv("WEEKLY_TAXES_CO_WITHOLDING"))
monthly_taxes_co_witholding = weekly_taxes_co_witholding * 4

In [7]:
# Taxes Group
taxes_color = "red"

# Add the node labels
add_node_label(node_df, label="Taxes", color=taxes_color)
add_node_connection(
    node_df,
    link_df,
    source="Income",
    target="Taxes",
    value=monthly_taxes,
)

# add_node_label(node_df, label="Taxes [Federal Witholding]", color=taxes_color)
# add_node_connection(
#     node_df,
#     link_df,
#     source="Taxes",
#     target="Taxes [Federal Witholding]",
#     value=monthly_taxes_fed_witholding,
# )

# add_node_label(node_df, label="Taxes [Federal FICA]", color=taxes_color)
# add_node_connection(
#     node_df,
#     link_df,
#     source="Taxes",
#     target="Taxes [Federal FICA]",
#     value=monthly_taxes_fed_fica,
# )

# add_node_label(node_df, label="Taxes [Federal OASDI]", color=taxes_color)
# add_node_connection(
#     node_df,
#     link_df,
#     source="Taxes",
#     target="Taxes [Federal OASDI]",
#     value=monthly_taxes_fed_oasdi,
# )

# add_node_label(node_df, label="Taxes [CO Parental Leave]", color=taxes_color)
# add_node_connection(
#     node_df,
#     link_df,
#     source="Taxes",
#     target="Taxes [CO Parental Leave]",
#     value=monthly_taxes_co_leave,
# )

# add_node_label(node_df, label="Taxes [CO Witholding]", color=taxes_color)
# add_node_connection(
#     node_df,
#     link_df,
#     source="Taxes",
#     target="Taxes [CO Witholding]",
#     value=monthly_taxes_co_witholding,
# )

## Benefits

In [8]:
weekly_dental = float(os.getenv("WEEKLY_DENTAL"))
monthly_dental = weekly_dental * 4

weekly_vision = float(os.getenv("WEEKLY_VISION"))
monthly_vision = weekly_vision * 4


### Retirement and Savings

In [9]:
weekly_401k_pretax = float(os.getenv("WEEKLY_401K_PRETAX")) # before tax
monthly_401k_pretax = weekly_401k_pretax * 4

weekly_401k_roth = float(os.getenv("WEEKLY_401K_ROTH"))  # after tax
monthly_401k_roth = weekly_401k_roth * 4

monthly_401k = monthly_401k_pretax + monthly_401k_roth

# accumulate the net benefits value
monthly_benefits = sum([
    monthly_dental, monthly_vision, monthly_401k,
])

In [10]:
weekly_roth_ira = float(os.getenv("WEEKLY_ROTH_IRA")) # after tax
monthly_roth_ira = weekly_roth_ira * 4  # after tax

weekly_savings_hysa = float(os.getenv("WEEKLY_SAVINGS_HYSA"))  # big nose max -- just whatever i don't spend
monthly_savings_hysa = weekly_savings_hysa * 4

# accumulate the net savings (post-tax and benefits)
monthly_savings_net = sum([
    monthly_roth_ira, monthly_savings_hysa]
)

In [11]:
benefits_color = "orange"

# Add the node labels
add_node_label(node_df, label="Benefits", color=benefits_color)
add_node_connection(
    node_df,
    link_df,
    source="Income",
    target="Benefits",
    value=monthly_benefits,
)

add_node_label(node_df, label="Dental", color=benefits_color)
add_node_connection(
    node_df,
    link_df,
    source="Benefits",
    target="Dental",
    value=monthly_dental,
)

add_node_label(node_df, label="Vision", color=benefits_color)
add_node_connection(
    node_df,
    link_df,
    source="Benefits",
    target="Vision",
    value=monthly_vision,
)

In [12]:
# 401k group
savings_color = "green"

add_node_label(node_df, label="401k", color=savings_color)
add_node_connection(
    node_df,
    link_df,
    source="Benefits",
    target="401k",
    value=monthly_401k,
)

add_node_label(node_df, label="401k Pre-tax", color=savings_color)
add_node_connection(
    node_df,
    link_df,
    source="401k",
    target="401k Pre-tax",
    value=monthly_401k_pretax,
)

add_node_label(node_df, label="401k Roth", color=savings_color)
add_node_connection(
    node_df,
    link_df,
    source="401k",  # comes out post-tax!
    target="401k Roth",
    value=monthly_401k_roth,
)

add_node_label(node_df, label="Net Savings", color=savings_color)
add_node_connection(
    node_df,
    link_df,
    source="Net Income",
    target="Net Savings",
    value=monthly_savings_net,
)

add_node_label(node_df, label="HYSA", color=savings_color)
add_node_connection(
    node_df,
    link_df,
    source="Net Savings",
    target="HYSA",
    value=monthly_savings_hysa,
)

add_node_label(node_df, label="Roth IRA", color=savings_color)
add_node_connection(
    node_df,
    link_df,
    source="Net Savings",  # comes out post-tax!
    target="Roth IRA",
    value=monthly_roth_ira,
)

## Expenses

In [13]:
monthly_gas = float(os.getenv("MONTHLY_GAS"))

weekly_dates = float(os.getenv("WEEKLY_DATES"))  # for katelyn and i (includes food (not double counting))
monthly_dates = weekly_dates * 4

weekly_food = float(os.getenv("WEEKLY_FOOD"))  # for myself or others
monthly_food = weekly_food * 4

monthly_misc = float(os.getenv("MONTHLY_MISC"))  # hard to estimate

In [14]:
# TODO: fill out estimated expenses
expenses_color = "yellow"
subscriptions_color = "purple"

# Expenses group
add_node_label(node_df, label="Gas", color=expenses_color)
add_node_connection(
    node_df,
    link_df,
    source="Net Income",
    target="Gas",
    value=monthly_gas,
)

add_node_label(node_df, label="Dates", color=expenses_color)
add_node_connection(
    node_df,
    link_df,
    source="Net Income",
    target="Dates",
    value=monthly_dates,
)

add_node_label(node_df, label="Misc", color=expenses_color)
add_node_connection(
    node_df,
    link_df,
    source="Net Income",
    target="Misc",
    value=monthly_misc,
)

add_node_label(node_df, label="Food", color=expenses_color)
add_node_connection(
    node_df,
    link_df,
    source="Net Income",
    target="Food",
    value=monthly_food,
)

### Subscriptions

In [15]:
monthly_icloud = float(os.getenv("MONTHLY_ICLOUD"))
monthly_audible = float(os.getenv("MONTHLY_AUDIBLE"))

monthly_subscriptions = sum(
    [
        monthly_icloud,
        monthly_audible,
    ]
)

In [16]:
add_node_label(node_df, label="Subscriptions", color=subscriptions_color)
add_node_connection(
    node_df,
    link_df,
    source="Net Income",
    target="Subscriptions",
    value=monthly_subscriptions,
)

add_node_label(node_df, label="iCloud", color=subscriptions_color)
add_node_connection(
    node_df,
    link_df,
    source="Subscriptions",
    target="iCloud",
    value=monthly_icloud,
)

add_node_label(node_df, label="Audible", color=subscriptions_color)
add_node_connection(
    node_df,
    link_df,
    source="Subscriptions",
    target="Audible",
    value=monthly_audible,
)

## Plot

In [17]:
fig = go.Figure(
    data=[
        go.Sankey(
            orientation="h",
            valueformat=".2f",
            arrangement="snap",
            node=dict(
                pad=10,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=node_df["Label"],
                color=node_df["Color"],
            ),
            link=dict(
                source=link_df["Source"],
                target=link_df["Target"],
                value=link_df["Value"],
            ),
        )
    ]
)

height = 900
width = height * 1.5

# TODO: add legend for colors?
fig.update_layout(
    title_text=f"""
    Monthly Income=${monthly_salary:,.2f}; Annual=${weekly_salary*52:,.2f}
    """,
    font_size=12,
    height=height,
    width=width,
)

fig.show()

## Save the contents

In [18]:
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d_%H")  # Format: YYYYMMDD_HH

# Construct the filename
base = Path("images") / timestamp
base.mkdir(exist_ok=True, parents=True)

filename = base / "sankey_chart"

# Save the chart as an SVG file
fig.write_html(filename.with_suffix(".html"))
fig.write_image(filename.with_suffix(".svg"))
fig.write_image(filename.with_suffix(".png"))

print(f"Chart saved as: {filename}")

Chart saved as: images/20250331_16/sankey_chart
